In [16]:
!git clone https://github.com/franciscogarate/utils_ucm

fatal: destination path 'utils_ucm' already exists and is not an empty directory.


In [17]:
!pip install pyliferisk


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np

In [19]:
from utils_ucm import clr

In [20]:
from pyliferisk import MortalityTable, lx
from pyliferisk.mortalitytables import PASEM2020_Decesos_M_2ord
mt = MortalityTable(qx=PASEM2020_Decesos_M_2ord)

In [21]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

In [22]:
escenario = 'Base'
shock_mort = False
shock_c_up = False
shock_c_do = False
shock_c_ma = False
shock_gast = False
shock_cat_ = False

In [23]:
edad = 50
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(mt.w -edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: mt.lx[x+1] if x <= mt.w else 0)
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df['capital'] = incr_capital(5000, df.t)
df['pagos'] = df['capital'] * df['qx']
df['caida'] = 0.03
df['polizas'] = (1 - df['caida']).cumprod()
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df['polizas_benef'] =  df['px'] * df['polizas']
df['pagos_prob'] = df['pagos'] * df['polizas_benef']
df['gastos'] = df['capital'] * 0.01
df['gastos_prob'] = df['gastos'] * df['polizas_benef']
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
#df.salidas_prob @ df.factor_desc

In [25]:
df.to_excel('Flujos2.xlsx', sheet_name=f'{escenario}')
